In [2]:
import numpy as np 
import pandas as pd
import os
import re
import warnings
#import pandas_profiling
warnings.filterwarnings('ignore')

In [3]:
raw_data= pd.read_csv('play_raw_data.csv')
raw_data.head()


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [25]:
# import pandas_profiling
#pandas_profiling.ProfileReport(raw_data)

In [4]:
raw_data.shape

(10841, 13)

In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


# Data Cleaning

The size of the raw data is 10841x13 with 1487 missing values, and 483 duplicates.
after cleaning, we get a new dataset 10829x13 which we will work on consists of:
   ## * Categorical veribles: 
App, Category, type, Conten Rating, Genres, last Update, current ver, min_andriod_version
   ## * Real number:
Rating, Size, Install_num, price_usd, previews_count

In [6]:
raw_data.isna().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [7]:
# the 1474 missing values in Rating are replaced by the mean of the Rating
raw_data['Rating'].fillna(raw_data['Rating'].mean(), inplace= True)
raw_data.isna().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              1
Price             0
Content Rating    1
Genres            0
Last Updated      0
Current Ver       8
Android Ver       3
dtype: int64

In [8]:
# droping the rest of missing
df = raw_data.dropna()
df.isna().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
dtype: int64

In [9]:
df.shape

(10829, 13)

In [10]:
# checking for duplicated apps
df[df.duplicated(['App'])]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
229,Quick PDF Scanner + OCR FREE,BUSINESS,4.2,80805,Varies with device,"5,000,000+",Free,0,Everyone,Business,"February 26, 2018",Varies with device,4.0.3 and up
236,Box,BUSINESS,4.2,159872,Varies with device,"10,000,000+",Free,0,Everyone,Business,"July 31, 2018",Varies with device,Varies with device
239,Google My Business,BUSINESS,4.4,70991,Varies with device,"5,000,000+",Free,0,Everyone,Business,"July 24, 2018",2.19.0.204537701,4.4 and up
256,ZOOM Cloud Meetings,BUSINESS,4.4,31614,37M,"10,000,000+",Free,0,Everyone,Business,"July 20, 2018",4.1.28165.0716,4.0 and up
261,join.me - Simple Meetings,BUSINESS,4.0,6989,Varies with device,"1,000,000+",Free,0,Everyone,Business,"July 16, 2018",4.3.0.508,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10715,FarmersOnly Dating,DATING,3.0,1145,1.4M,"100,000+",Free,0,Mature 17+,Dating,"February 25, 2016",2.2,4.0 and up
10720,Firefox Focus: The privacy browser,COMMUNICATION,4.4,36981,4.0M,"1,000,000+",Free,0,Everyone,Communication,"July 6, 2018",5.2,5.0 and up
10730,FP Notebook,MEDICAL,4.5,410,60M,"50,000+",Free,0,Everyone,Medical,"March 24, 2018",2.1.0.372,4.4 and up
10753,Slickdeals: Coupons & Shopping,SHOPPING,4.5,33599,12M,"1,000,000+",Free,0,Everyone,Shopping,"July 30, 2018",3.9,4.4 and up


In [11]:
df.sort_values(by=['Reviews'], inplace =True)

In [12]:
#dropping the duplicates based on the app name
df = df.drop_duplicates(keep='last', subset='App')

In [13]:
# preprocessing Installs column a& converting into float num
df['Installs'].unique()

array(['1,000+', '500+', '100+', '50+', '1+', '5+', '10+', '10,000+',
       '0+', '50,000+', '100,000+', '5,000+', '1,000,000+', '5,000,000+',
       '10,000,000+', '100,000,000+', '50,000,000+', '500,000+',
       '1,000,000,000+', '500,000,000+'], dtype=object)

In [14]:
df['Install_num'] = df['Installs'].apply(lambda x: float(x.split('+')[0].replace(',','')))
df['Install_num'].unique()

array([1.e+03, 5.e+02, 1.e+02, 5.e+01, 1.e+00, 5.e+00, 1.e+01, 1.e+04,
       0.e+00, 5.e+04, 1.e+05, 5.e+03, 1.e+06, 5.e+06, 1.e+07, 1.e+08,
       5.e+07, 5.e+05, 1.e+09, 5.e+08])

In [15]:
# preprocessing price column & con´verting into USD
df['Price'].unique()

array(['0', '$2.99', '$3.99', '$1.49', '$0.99', '$1.99', '$13.99',
       '$25.99', '$2.00', '$2.49', '$39.99', '$19.99', '$5.99', '$4.99',
       '$28.99', '$399.99', '$394.99', '$200.00', '$30.99', '$3.61',
       '$8.99', '$109.99', '$1.04', '$154.99', '$1.96', '$14.99', '$6.49',
       '$5.49', '$4.85', '$3.49', '$29.99', '$3.04', '$12.99', '$3.88',
       '$74.99', '$15.99', '$7.99', '$9.99', '$9.00', '$4.60', '$6.99',
       '$16.99', '$8.49', '$1.61', '$37.99', '$4.49', '$4.84', '$2.59',
       '$1.20', '$3.28', '$1.00', '$11.99', '$10.99', '$3.95', '$24.99',
       '$14.00', '$10.00', '$2.50', '$3.90', '$1.76', '$46.99', '$33.99',
       '$389.99', '$4.59', '$400.00', '$17.99', '$4.77', '$4.80',
       '$19.90', '$2.60', '$1.26', '$7.49', '$1.97', '$2.95', '$15.46',
       '$2.56', '$379.99', '$299.99', '$18.99', '$3.08', '$1.75', '$5.00',
       '$19.40', '$2.90', '$1.70', '$89.99', '$3.02', '$1.59', '$1.29',
       '$1.50', '$4.29', '$79.99'], dtype=object)

In [16]:
df['Price_usd'] = df['Price'].apply(lambda x: float(x.replace('$','')))
df['Price_usd'].unique()

array([  0.  ,   2.99,   3.99,   1.49,   0.99,   1.99,  13.99,  25.99,
         2.  ,   2.49,  39.99,  19.99,   5.99,   4.99,  28.99, 399.99,
       394.99, 200.  ,  30.99,   3.61,   8.99, 109.99,   1.04, 154.99,
         1.96,  14.99,   6.49,   5.49,   4.85,   3.49,  29.99,   3.04,
        12.99,   3.88,  74.99,  15.99,   7.99,   9.99,   9.  ,   4.6 ,
         6.99,  16.99,   8.49,   1.61,  37.99,   4.49,   4.84,   2.59,
         1.2 ,   3.28,   1.  ,  11.99,  10.99,   3.95,  24.99,  14.  ,
        10.  ,   2.5 ,   3.9 ,   1.76,  46.99,  33.99, 389.99,   4.59,
       400.  ,  17.99,   4.77,   4.8 ,  19.9 ,   2.6 ,   1.26,   7.49,
         1.97,   2.95,  15.46,   2.56, 379.99, 299.99,  18.99,   3.08,
         1.75,   5.  ,  19.4 ,   2.9 ,   1.7 ,  89.99,   3.02,   1.59,
         1.29,   1.5 ,   4.29,  79.99])

In [17]:
df['Reviews'].unique()

array(['0', '1', '10', ..., '9975', '999', '9992'], dtype=object)

In [18]:
# converting reviews num inton int
df['Reviews_count'] = df['Reviews'].apply(lambda x: int(x))
df['Reviews_count'].unique()

array([   0,    1,   10, ..., 9975,  999, 9992])

In [19]:
# preprocessing size column & converting into int
df['Size'].unique()

array(['10.0M', '4.1M', '29M', '18M', '9.1M', '14M', 'Varies with device',
       '4.3M', '20M', '28M', '4.2M', '3.2M', '5.8M', '3.7M', '7.9M',
       '4.9M', '1.7M', '1.6M', '15M', '22M', '56M', '1.1M', '26M', '11M',
       '4.6M', '3.3M', '18k', '3.6M', '4.0M', '13M', '2.1M', '27M', '47M',
       '5.9M', '118k', '2.9M', '45M', '43M', '5.2M', '52M', '12M', '17M',
       '7.7M', '6.0M', '9.7M', '9.9M', '3.4M', '1.4M', '3.9M', '8.6M',
       '10M', '2.8M', '24M', '2.3M', '16M', '35M', '30M', '25M', '7.0M',
       '32M', '2.7M', '475k', '8.4M', '54M', '3.5M', '49M', '7.4M',
       '6.9M', '6.6M', '42M', '8.7M', '19M', '3.0M', '1.3M', '2.5M',
       '4.8M', '4.5M', '36M', '1.2M', '39M', '7.5M', '4.4M', '21M',
       '7.8M', '3.8M', '7.6M', '38M', '2.6M', '8.2M', '8.0M', '92M',
       '2.4M', '8.9M', '5.3M', '9.8M', '5.0M', '40M', '90M', '7.3M',
       '55M', '688k', '1.8M', '9.0M', '68M', '269k', '5.7M', '2.2M',
       '44M', '940k', '642k', '9.4M', '5.5M', '9.3M', '6.2M', '48M',
       '

In [20]:
df['Size'].replace('Varies with device',np.nan,inplace=True)

In [21]:
df["Size"] = (df["Size"].replace(r'[kM]+$', '', regex=True).astype(float) 
               * df["Size"].str.extract(r'[\d\.]+([kM]+)',expand=False).fillna(1)
               .replace(["k","M"], [10**3, 10**6]).astype(int))


In [22]:
# replav´cing the nan with the mean of the size
df['Size'].fillna((df['Size'].mean()), inplace=True)

In [23]:
df['Size'].unique()

array([1.00000000e+07, 4.10000000e+06, 2.90000000e+07, 1.80000000e+07,
       9.10000000e+06, 1.40000000e+07, 2.04117135e+07, 4.30000000e+06,
       2.00000000e+07, 2.80000000e+07, 4.20000000e+06, 3.20000000e+06,
       5.80000000e+06, 3.70000000e+06, 7.90000000e+06, 4.90000000e+06,
       1.70000000e+06, 1.60000000e+06, 1.50000000e+07, 2.20000000e+07,
       5.60000000e+07, 1.10000000e+06, 2.60000000e+07, 1.10000000e+07,
       4.60000000e+06, 3.30000000e+06, 1.80000000e+04, 3.60000000e+06,
       4.00000000e+06, 1.30000000e+07, 2.10000000e+06, 2.70000000e+07,
       4.70000000e+07, 5.90000000e+06, 1.18000000e+05, 2.90000000e+06,
       4.50000000e+07, 4.30000000e+07, 5.20000000e+06, 5.20000000e+07,
       1.20000000e+07, 1.70000000e+07, 7.70000000e+06, 6.00000000e+06,
       9.70000000e+06, 9.90000000e+06, 3.40000000e+06, 1.40000000e+06,
       3.90000000e+06, 8.60000000e+06, 2.80000000e+06, 2.40000000e+07,
       2.30000000e+06, 1.60000000e+07, 3.50000000e+07, 3.00000000e+07,
      

In [24]:
# preprocessing the Android Ver column
df['Android Ver'].unique()

array(['4.0 and up', '4.0.3 and up', '4.3 and up', '4.1 and up',
       '4.2 and up', '2.3 and up', 'Varies with device', '5.0 and up',
       '4.4 and up', '6.0 and up', '1.6 and up', '2.1 and up',
       '5.1 and up', '2.2 and up', '2.3.3 and up', '3.0 and up',
       '3.2 and up', '4.4W and up', '1.5 and up', '3.1 and up',
       '5.0 - 7.1.1', '7.0 and up', '8.0 and up', '2.0.1 and up',
       '2.0 and up', '7.0 - 7.1.1', '4.0.3 - 7.1.1', '1.0 and up',
       '5.0 - 8.0', '7.1 and up', '5.0 - 6.0', '2.2 - 7.1.1',
       '4.1 - 7.1.1'], dtype=object)

In [25]:
df['Android Ver'].replace('Varies with device', np.nan, inplace = True)

In [26]:
#finding out the minimum android version supported for the apps

df['min_android_version'] = df['Android Ver'].apply(lambda x: re.sub('[a-zA-Z]','', str(x)))

In [27]:
df['min_android_version'].unique()

array(['4.0  ', '4.0.3  ', '4.3  ', '4.1  ', '4.2  ', '2.3  ', '',
       '5.0  ', '4.4  ', '6.0  ', '1.6  ', '2.1  ', '5.1  ', '2.2  ',
       '2.3.3  ', '3.0  ', '3.2  ', '1.5  ', '3.1  ', '5.0 - 7.1.1',
       '7.0  ', '8.0  ', '2.0.1  ', '2.0  ', '7.0 - 7.1.1',
       '4.0.3 - 7.1.1', '1.0  ', '5.0 - 8.0', '7.1  ', '5.0 - 6.0',
       '2.2 - 7.1.1', '4.1 - 7.1.1'], dtype=object)

In [28]:
df.isna().sum()

App                      0
Category                 0
Rating                   0
Reviews                  0
Size                     0
Installs                 0
Type                     0
Price                    0
Content Rating           0
Genres                   0
Last Updated             0
Current Ver              0
Android Ver            989
Install_num              0
Price_usd                0
Reviews_count            0
min_android_version      0
dtype: int64

In [29]:
# dropping the unwanted columns
data = df.drop(['Reviews','Installs','Price','Android Ver'],axis='columns')
data.head()

,App,Category,Rating,Size,Type,Content Rating,Genres,Last Updated,Current Ver,Install_num,Price_usd,Reviews_count,min_android_version
5086,Apps for SportsBєtting.ag - Bitcoin Welcome here!,SPORTS,4.193338,10000000.0,Free,Everyone,Sports,"December 11, 2017",1.0,1000.0,0.0,0,4.0
7728,HR Team CQ Region Ed Qld,BUSINESS,4.193338,4100000.0,Free,Everyone,Business,"January 12, 2018",3.8,500.0,0.0,0,4.0.3
5165,Ferguson AH,MEDICAL,4.193338,29000000.0,Free,Everyone,Medical,"July 17, 2017",300000.0.86,100.0,0.0,0,4.0.3
5162,Hilltop AH,MEDICAL,4.193338,29000000.0,Free,Everyone,Medical,"January 10, 2018",300000.0.96,100.0,0.0,0,4.0.3
5161,Bayview Hill AH,MEDICAL,4.193338,29000000.0,Free,Everyone,Medical,"June 6, 2018",300000.1.08,50.0,0.0,0,4.0.3


In [30]:
# show a general summary of the data
pandas_profiling.ProfileReport(data)

NameError: name 'pandas_profiling' is not defined